
## Open Source Tools For Rapid Statistical Model Development

### Overview
* Catching up with Probabilistic Programming
* Plan models -- causalgraphicalmodels
* Loading Data -- PANDAS
* Preparing Data -- PANDAS, Seaborn, Scikit-learn
* Rapid Model Development -- PyMC3
* Beyond the Basics

<a id='TOP'></a>
### Notebook Content:
1. [PP Overview](#Overview)
    1. PP as modeling framework
    2. Emphasis on generalizing predictive models
1. [Plan modeling](#ModelPlan)
2. [Loading Field Data](#DataLoad)
3. [Prepare Data for Modeling](#DataPrep)
4. [Bayesian Modelling](#PyMC3)
   1. [Model coding](#writemodel)
   2. [Prior evaluation & Model modification](#priors)
   3. [Model fitting & diagnostics](#fit)
   4. [Model Evaluation](#eval)
       1. [PPC](#ppc)
       2. [Out-of-sample (test set) performance](#test)
           1. [Usual suspects; $r^2$, $mae$](#mae)
           2. [Unusual suspect: $deviance$](#deviance)
           3. [Approximating test set performance; WAIC](#waic)
5. [Conclusion](Conclusion)

In [27]:
# STL
import sys

# numerical libraries
import numpy as np
import pandas as pd
import pymc3 as pm

# graphic facilities
from matplotlib import rcParams
import matplotlib.pyplot as pl
import seaborn as sb

In [3]:
rcParams['axes.formatter.limits'] = (-2, 3)
rcParams['axes.titlesize'] = 18
rcParams['axes.labelsize'] = 16
rcParams['font.size'] = 16
rcParams['ytick.labelsize'] = 16
rcParams['xtick.labelsize'] = 16
rcParams['legend.fontsize'] = 16
rcParams['xtick.minor.visible'] = True

In [4]:
print(f'Python: version {sys.version.split("|")[0]}')
print(f'PanDaS version {pd.__version__}')
print(f'Seaborn version {sb.__version__}')
print(f'PyMC3 version {pm.__version__}')

Python: version 3.7.1 
PanDaS version 0.24.1
Seaborn version 0.9.0
PyMC3 version 3.6


In [25]:
%matplotlib inline

### Loading and preparing data -- PANDAS
* the nomad dataset
* reading in 
* get column names
* extract desired variables

### Data Exploration -- PANDAS, Seaborn and Scikit-Learn
* predictor isolated distributions
* plotting predictors/predicted w/ respect to each other
* predictor correlation, multicollinearity and pca

### Modeling -- Probabilistic Programming with PyMC3
* simple bayesian regression to predict chlorophyll from Rrs
* rapid but transparent model development
* evaluation of priors
* fitting and evaluation of posterior distribution
* model comparison/selection

<a id=DataLoad></a>
## <center><u>Loading Field Data</u></center> 
<br>
<span style="font-size:16pt">$\ \ \ \Rightarrow$ NOMAD SeaWiFS validation match-up distribution</span><br>
<img src='./resources/nomad_seabass_v1_seawifs_map.png' width=600 title="NOMAD SeaWiFS validation match-up distribution"/>

<span style="font-size:16pt">$\Rightarrow$   <u>Predictors</u></span>
* sat_rrs  ~ satellite remote-sensing reflectance (sr-1)
* etopo2 ~ NGDC ETOPO2 water depth (meters)
* oisst    ~ optimal interpolation (OI) sea surface temperature (SST) (degrees C)
* senz     ~ satellite viewing (zenith) angle (degrees)
* solz     ~ solar zenith angle at time of satellite overpass (degrees)
---
<span style="font-size:16pt">$\Rightarrow$   <u>Predicted</u></span>
* chl ~ fluorometric chlorophyll a

<u>Other</u>
* id ~ NOMAD record identifier (unitless)

<span style="font-size:16pt">$\Rightarrow$   <u>File content</u></span>
<br><br>
/fields=year,month,day,hour,minute,second,lat,lon,id,oisst,etopo2,chl,chl_a,kd405,kd411,kd443,kd<br>
...<br>
_rrs490,sat_rrs510,sat_rrs555,sat_rrs670,sat_file,tdiff,solz,senz,cv,valid<br>
/units=-999<br>
/end_header<br>
1997,10,07,09,41,00,42.51,39.52,4065,19.54,1982,0.401,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,

In [7]:
with open('./nomad_seawifs_v1.3_2005262.txt') as f:
    for i, line in enumerate(f.readlines()):
        if 'fields' in line:
            # collect column labels
            col_names = line.strip('/fields=').strip().split(',') 
        if '/end_header' in line:
            # record columns to skip
            rowskips = i+1
            break

In [8]:
df = pd.read_csv('./nomad_seawifs_v1.3_2005262.txt', header=None,
                 names=col_names, skiprows=rowskips, na_values=-999)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Columns: 169 entries, year to valid
dtypes: float64(157), int64(10), object(2)
memory usage: 385.6+ KB


In [14]:
df.head()

,year,month,day,hour,minute,second,lat,lon,id,oisst,...,sat_rrs490,sat_rrs510,sat_rrs555,sat_rrs670,sat_file,tdiff,solz,senz,cv,valid
0,1997,10,7,9,41,0,42.5100,39.5200,4065,19.54,...,0.00345,0.00297,0.00217,0.00026,S1997280094210.L1A_GAC,480,49.0,25.4,0.2098,0
1,1997,10,11,9,32,0,39.2900,25.1100,4069,19.57,...,0.00494,0.00348,0.00191,0.00018,S1997284110315.L1A_MLAC,5880,48.7,37.8,0.1043,1
2,1997,9,27,11,29,0,24.1392,-20.9995,6083,24.67,...,0.00625,0.00375,0.00168,0.00022,S1997270134450.L1A_MLAC,8880,28.0,35.5,0.1012,1
3,1998,5,16,8,30,0,-32.3397,17.8766,6119,17.21,...,0.00177,0.00193,0.00196,0.00055,S1998136102611.L1A_MLAC,7200,51.6,22.4,0.0940,1
4,1998,5,16,9,16,0,-32.3447,17.8735,6120,17.21,...,0.00180,0.00198,0.00199,0.00056,S1998136102611.L1A_MLAC,4440,51.6,22.4,0.0836,1


In [ ]:
def Daylight(row):
    l = astral.Location()
    l.latitude = row['lat']
    l.longitude = row['lon']
    try:
        td = l.sunset(date=row['datetime']) - l.sunrise(date=row['datetime'])
        return td.seconds
    except:
        return np.NaN

<a id=DataPrep></a>
## <center><u>Data Preparation</u></center> 
<br>

In [15]:
date_time_columns = ['year', 'month', 'day', 'hour', 'minute', 'second']
df.insert(0, 'datetime', pd.to_datetime(df[date_time_columns],
                                        format='%Y-%m-%d %H:%M:%S'))

In [17]:
df.drop(date_time_columns, axis=1, inplace=True)

In [22]:
# two types of chlorophyll a:
df.filter(regex='chl', axis=1).info()
# chl: obtained by fluorometry
# chl_a: obtained by hplc

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 2 columns):
chl      262 non-null float64
chl_a    33 non-null float64
dtypes: float64(2)
memory usage: 4.6 KB


In [28]:
def fill_chl(row):
    # function to merge chlorophyll a columns into 1
    return row['chl_a'] if np.isfinite(row['chl_a']) else row['chl']
df['chlor_a'] = df.apply(fill_chl, axis=1) # merge chlorophyll a columns
df['is_hplc'] = np.isfinite(df['chl_a']) # mark chl measurement type as boolean flag

In [29]:
# Preprocess satellite data
df_sat = df.filter(regex='sat_rrs', axis=1).copy()
# Invalidate negative readings
df_sat[df_sat<=0] = np.NaN
# Log-transform rrs
df_log_sat = pd.DataFrame(np.log10(df_sat.values),
                          columns=[f'log_{col}' for col in df_sat.columns]
                         )

In [45]:
# Preprocess 
df_extr = df[['oisst']].merge(df_log_sat,
                                  right_index=True,
                                  left_index=True)
df.loc[df.etopo2==0, 'etopo2'] = np.NaN
df_extr.insert(1, 'log_depth', np.log10(df.etopo2))
df_extr.insert(1, 'coastal', df.etopo2<=30)
df_extr['coastal'] = df_extr.coastal.astype('category').cat.codes.astype('int')
df_extr['log_chlor_a'] = np.log10(df.chlor_a.values)

In [48]:
df_extr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 10 columns):
oisst             292 non-null float64
coastal           292 non-null int64
log_depth         291 non-null float64
log_sat_rrs412    276 non-null float64
log_sat_rrs443    285 non-null float64
log_sat_rrs490    291 non-null float64
log_sat_rrs510    292 non-null float64
log_sat_rrs555    292 non-null float64
log_sat_rrs670    281 non-null float64
log_chlor_a       292 non-null float64
dtypes: float64(9), int64(1)
memory usage: 22.9 KB


In [51]:
df_extr.coastal.value_counts()

0    198
1     94
Name: coastal, dtype: int64